<a href="https://colab.research.google.com/github/jansoe/KISchule/blob/main/A5_1_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. Automatische Textvergleiche (klassisch) - Teil 2

Das Verfahren ist wie gehabt:
- Erstellen Sie eine Kopie dieses Notebooks in ihrem Google Drive (vorgeschlagene Umbenennung: "A5_Teil1 - Vorname, Nachname")
- Editieren Sie die Text- und Codezellen.
- Schicken Sie uns einen Freigabelink zum Kommentieren Ihres Notebooks an

## 5.1. Arbeiten mit Daten von Wikipedia

### Allgemeine Vorbereitungen

Zunächst importieren wir alle Bibliotheken, die im Verlauf dieses Notebooks benötigt werden.


In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import json
import gensim
import time

Binden Sie nun Ihr Google Drive ein, damit Sie dort Daten ablegen und zu einem späteren Zeitpunkt (d.h. in einer anderen Session) auf diese erneut zugreifen können.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Ebenso wie im vorherigen Notebook benötigen wir für die anschließenden Schritte einen Tokenizer. Wir definieren dafür die gleiche Funktion wie schon zu Beginn von Abschnitt 5.1:

In [ ]:
def tokenizer(text):
    tokens = list(gensim.utils.simple_tokenize(text))
    return tokens

Schließlich wählen Sie ein geeignetes Verzeichnis Ihrer Wahl in Ihrem Google Drive aus (und erstellen dieses bei Bedarf), in das die für diesen Abschnitt benötigten Textdaten ablegt werden können.

Die drei Variablen `article_file`, `filtered_file` und `corpus_file` werden auch an dieser Stelle bereits definiert, damit Sie u.U. direkt zu Abschnitt 5.4.2 springen können.

In [ ]:
# Bitte die Variable path entsprechend anpassen
import os
path = "/content/drive/My Drive/KI-Schule/2 - Textdaten/wiki/"
os.makedirs(path)

# Definition von Dateinamen, die in 5.4.1 zum Speichern und in 5.4.2 zum Laden verwendet werden
article_file = path + "article_dict.json"
filtered_file = path + "wiki_filtered.dict"
corpus_file = path + "wiki_corpus.mm"

### Daten herunterladen und vorbereiten

Jetzt führen wir ein Skript aus, das einen Teil des sogenannten Wikipedia-Dumps (Rohdaten verschiedener Wikipedia-Artikel) in verwertbaren Text umwandelt, z.B. durch das Entfernen von HTML-Formatierungen. Dazu muss der entsprechende Dump zuerst heruntergeladen und in Ihrem Google Drive abgelegt werden.

Um die Daten ohne Umwege direkt in Google Drive kopieren zu können, kann das Programm `wget` verwendet werden. Sollten Sie Probleme mit dem Download bekommen, müssen Sie evtl. einen aktuellen Pfad für die Variable `link_address` angeben. An die Rohdaten von Wikipedia und somit die entsprechende Link-Adresse gelangen Sie z.B. über diesen [LINK](https://dumps.wikimedia.org/dewiki/latest/).

**Anmerkungen**

- **Achtung: Das Ausführen der Code-Zellen dieses Abschnitts benötigt sehr viel Zeit!**
- Da Sie die Ergebnisse in Ihrem Google Drive abspeichern werden, sollten Sie die zugehörigen Befehle jedoch lediglich einmal ausführen müssen. Anschließend können Sie auch nach einer Bearbeitungspause nach 5.4.0 direkt mit 5.4.2 weitermachen.
- Wir empfehlen Ihnen, alle Schritte dieses Abschnitts "in einem Rutsch" (in einer Session) abzuarbeiten. Rechnen Sie hierfür mit einem ungefähren Zeitaufwand von 1h.

In [ ]:
link_address = "https://dumps.wikimedia.org/dewiki/latest/dewiki-latest-pages-articles-multistream1.xml-p1p262467.bz2"
dump_file = path + "wiki_dump.bz2"
!wget "{link_address}" -O "{dump_file}"

--2020-01-25 15:24:43--  https://dumps.wikimedia.org/dewiki/latest/dewiki-latest-pages-articles-multistream1.xml-p1p262467.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 469739844 (448M) [application/octet-stream]
Saving to: ‘/content/drive/My Drive/KI-Schule/2 - Textdaten/wiki/wiki_dump.bz2’

/content/drive/My D 100%[===================>] 447.98M  5.02MB/s    in 89s     

2020-01-25 15:26:13 (5.02 MB/s) - ‘/content/drive/My Drive/KI-Schule/2 - Textdaten/wiki/wiki_dump.bz2’ saved [469739844/469739844]



Jetzt können wir das angesprochene Skript anwenden, dass aus den Rohdaten die verwertbaren Bestandteile extrahiert.

In [ ]:
processed_file = path + "wiki-latest.json.bz"
!python -m gensim.scripts.segment_wiki -i -f "{dump_file}" -o "{processed_file}"

Als nächstes wandeln wir den extrahierten Text in ein Dictionary um, das die jeweiligen Titel als Schlüssel (keys) und die zugehörigen Dokumente als Werte (values) beinhaltet.

In [ ]:
processed_file = path + "wiki-latest.json.bz"
article_dict = {}
with gensim.utils.smart_open(processed_file, 'rb') as f:
    for line in f:
        article = json.loads(line)

        article_dict[article['title']] = '\n'.join(article['section_texts'])
# Ergebnis abspeichern
json.dump(article_dict, open(article_file, 'w'))

Nun erstellen wir wie schon in Abschnitt 5.1 mit Hilfe des Tokenizers das Token Dictionary.

In [ ]:
token_dictionary = gensim.corpora.Dictionary()
cache = []
to_process = len(article_dict)
for text in article_dict.values():
    if to_process % 1000 == 0:
        # Füge Cache dem Dictionary hinzu
        token_dictionary.add_documents(cache)
        cache = []
        clear_output()
        print(f'{to_process} documents still need processing')
    tokenized_text = tokenizer(text)
    cache.append(tokenized_text)
    to_process -= 1
# Ergebnis abspeichern
wiki_dict_file = path + "wiki.dict"
token_dictionary.save(wiki_dict_file)
clear_output()

Als nächstes filtern wir unser Token Dictionary ...

In [ ]:
token_dictionary = gensim.corpora.Dictionary.load(wiki_dict_file)
token_dictionary.filter_extremes(
    no_below=5,     # wir filtern alle Tokens, die seltener als X mal vorkommen 
    no_above=0.7,   # wir filtern alle Tokens, die in mehr als X Prozent vorkommen
    keep_n=500000   # wir behalten maximal 500.000 tokens
)
# Ergebnis abspeichern
token_dictionary.save(filtered_file)

und erstellen schließlich unseren finalen "vektorisierten" Korpus.

In [ ]:
corpus_numeric = []
to_process = len(article_dict)
for text in article_dict.values():
    if to_process % 1000 == 0:
        clear_output()
        print(f'{to_process} documents still need processing')
    tokenized_text = tokenizer(text)
    corpus_numeric.append(token_dictionary.doc2bow(tokenized_text))
    to_process -= 1
# Ergebnis abspeichern
gensim.corpora.MmCorpus.serialize(corpus_file, corpus_numeric)

### Ähnlichkeit von Wikipedia-Artikeln

**Anmerkungen**

Haben Sie den Abschnitt **Daten herunterladen und vorbereiten** bereits zu einem früheren Zeitpunkt (d.h. in einer anderen Sesssion) ausgeführt, müssen Sie darauf achten, dass Sie nun zunächst die Code-Zellen des Abschnitts **Allgemeine Vorbereitungen** ausführen, um alle benötigten Bibliotheken zu importieren sowie den `Tokenizer` und die Variable `path` zu definieren.

In [ ]:
token_dictionary = gensim.corpora.Dictionary.load(filtered_file)
corpus_numeric = gensim.corpora.MmCorpus(corpus_file)
article_dict = json.load(open(article_file))

titles = list(article_dict.keys())

Als eine aussagekräftige numerische Repräsentation verwenden wir das im letzten Notebook vorgestellte TFIDF-Verfahren.

In [ ]:
tfidf = gensim.models.TfidfModel(corpus_numeric)

Das anschließende Berechnen der paarweisen Ähnlichkeiten nimmt ein paar Minuten in Anspruch.

In [ ]:
index_wiki = gensim.similarities.SparseMatrixSimilarity(tfidf[corpus_numeric], num_features=corpus_numeric.num_terms)

Die großen Vorverarbeitungsschritte sind hiermit abgeschlossen. Nun können wir  Ergebnisse zu verschiedenen Arten von Fragestellungen errechnen lassen, ohne dabei lange Wartezeiten in Anspruch nehmen zu müssen.

#### Auffinden von verwandten Artikel zu einem ausgewählten Artikel

Zum Beispiel können wir einen bestimmten Artikel auswählen ...

In [ ]:
# In unserem Fall sollte der Index zum Turing-Test-Artikel führen
doc_number = 19856
print(titles[doc_number])

... und uns die zehn ähnlichsten Artikel (basierend auf dem TF/IDF-Ähnlichkeitsmaß) des Korpus ausgeben lassen.

In [ ]:
doc_vector = corpus_numeric[doc_number]

index_wiki.num_best = 10
match = index_wiki[tfidf[doc_vector]]

print('Der Artikel', titles[doc_number], 'ähnlich zu:')
for ix, score in match:
    print(titles[ix], f'{score:.2f}')

#### Auffinden von passenden Artikeln zu einem Suchwort

In diesem Beispiel starten wir mit einem Begriff, um uns die hierzu passensten Artikel heraussuchen zu lassen.

In [ ]:
query =  'Intelligenz' 
index_wiki.num_best = 2000

query_vector = token_dictionary.doc2bow(tokenizer(query))

match = index_wiki[query_vector]
match_index = [i[0] for i in match]

print([titles[ix] for ix in match_index])
print(len(match_index), 'matches')


Nun wollen wir anschauen, wie die Artikel, die wir aus unserem Query enthalten haben, untereinander verwandt sind. Hierfür berechnen wir zunächst die Ähnlichkeitsmaße sämtlicher paarweisen Vergleiche der oben ermittelten 2000 Artikel.

In [ ]:
index_wiki.num_best = None # in diesem Fall ohne Beschränkung der Anzahl ähnlicher Artikel
all2all_similarities = index_wiki[tfidf[corpus_numeric[match_index]]]

### Visualisieren von Artikelähnlichkeiten mittels t-SNE

Um diese hohe Anzahl an Dokumentenähnlichkeiten zu veranschaulichen, benutzen wir eine `unsupervised learning` Technik namens t-SNE (T-distributed Stochastic Neighbor Embedding). t-SNE wird insbesondere dafür verwendet, hochdimensionale Daten in zwei oder drei Dimensionen zu visualisieren. 

Vereinfacht gesprochen versucht t-SNE Nachbarschaften eines hochdimensionalen Datensatzes in einer niedrigdimensionalen Repräsentation korrekt abzubilden. Das bedeutet, dass die Distanzen zwischen Punkten, die in der hochdimensionalen Repräsentation nahe beieinander liegen, auch in der niedrigdimensionalen Darstellung möglichst genauso beibehalten werden.

**Anmerkungen**

Eine detaillierte Erklärung zu t-SNE gibt es [hier](https://www.youtube.com/watch?v=NEaUSP4YerM).

In [ ]:
from sklearn.manifold import TSNE
# Die Perplexity gibt an, was als "Nachbarschaft" zählt 
# (je größer, desto mehr Einfluss haben auch weiter entfernte Punkte)
tsne = TSNE(
    init='pca', 
    perplexity=10,     
    learning_rate=200  
)
tsne_vis = tsne.fit_transform(all2all_similarities[:, match_index])

Für die eigentliche Visualisierung verwenden wir in diesem Fall die Bibliotheken `plotly` und `pandas`.

In [ ]:
import plotly.express as px
import pandas as pd

to_plot = pd.DataFrame(tsne_vis, columns=['x', 'y'])
to_plot['labels'] = np.array(titles)[match_index]

Zunächst können wir einen Blick auf die grundsätzliche Struktur der Daten werden. Hier sehen wir, dass jedem Artikel eine zweidimensionale Koordinate (x, y) zugewiesen wurde.

In [ ]:
to_plot.head()

Schließlich lassen wir uns einen Scatterplot erstellen, in dem es einige interessante Zusammenstellungen lokaler Cluster zu entdecken gilt.

In [ ]:
px.scatter(to_plot, x='x', y='y', hover_name='labels')

### 5.1.0 Gemeinsamkeiten von Clustern

Suchen Sie sich drei lokale Cluster heraus (z.B. mit Bezug zum Thema KI) und versuchen Sie durch Lesen der entsprechenden Wikipedia-Artikel das gemeinsame Thema bzw. die wesentlichen Gemeinsamkeiten zu identifizieren und hier zu beschreiben.


<bitte ausfüllen>

### 5.1.1 High Perplexity
Stellen Sie den t-SNE Parameter `perplexity` auf 200 und visualisieren Sie die zugehörigen Nachbarschaftsverhältnisse.

Welche Unterschiede zur ursprünglichen Darstellen beobachten Sie und wie können sie diese erklären?

<bitte ausfüllen>

### 5.1.2 Low Perplexity
Setzen Sie den Parameter `perplexity` auf 1 und visualisieren Sie die zugehörigen Nachbarschaftsverhältnisse.

Welche Unterschiede zur ursprünglichen Darstellen beobachten Sie und wie können sie diese erklären?

<bitte ausfüllen>

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)